Table of content 
- [Introduction](#1)
- [The Halftoning Algorithm](#2)
  

<h1 style="color:salmon" id="1">Introuduction</h1>

-  Black and white printers put gray shade images on paper via the presence or absence of black ink on paper.
-  Printers do not contain ink with varying shades of gray.
-  These black and white output devices create the <span style="color:red">**illusion of shades of gray via haltoning.**</span>
-  Halftoning : converts an image of gray scale $\rarr$ to an image containing 0s and 1s only.
   -  halftoning breaks down an image into a grid of dots. 
   -  The human eye blends these dots to perceive a continuous image with gradients and shades
      -  even though it’s actually composed of discrete dots.
-  Dots that make optical illusion of shades and gradients, these dots may vary in the size.
- Applications 
  - Print Media 🖨️ 
  - Digital Displays (e-ink)
>Halftoning is essential for reproducing images where continuous tone is impossible, and it provides a balance between quality and resource efficiency, especially in printing.

>halftone process reduces visual reproductions to an image that is printed with only one color of ink, in dots of differing size (pulse-width modulation) or spacing (frequency modulation) or both.

<h1 style="color:salmon" id="2">The Halftoning Algorithm</h1>

- The basis of the halftoning algorithm is an **Error Diffusion Technique** 
- When the error reach a certin value, turn a pixel on and reset the error.
- if the error not great enough leave the pixel truned off.
- Error resulting from approximating a gray shade image with only ones and zeros.
  
```  
1. For each pixel in the image (left to right, top to bottom):
    a. Calculate the "old_pixel" value (current grayscale value of the pixel, between 0 and 255).
    b. Determine the "new_pixel" value by setting it to either black (0) or white (255):
       - If old_pixel >= 128, set new_pixel to 255 (white).
       - Otherwise, set new_pixel to 0 (black).
       
    c. Calculate the "quantization error":
       - error = old_pixel - new_pixel

    d. Diffuse the error to neighboring pixels that haven’t been processed yet:
       - Distribute the error to adjacent pixels using Floyd-Steinberg weights:
         - Pixel to the right: image[x + 1][y]   += error * 7/16
         - Pixel to the bottom-left: image[x - 1][y + 1] += error * 3/16
         - Pixel directly below: image[x][y + 1] += error * 5/16
         - Pixel to the bottom-right: image[x + 1][y + 1] += error * 1/16

2. Repeat for all pixels in the image.
```

